In [9]:
import torch
import numpy as np
import click
import dnnlib
import os
import PIL.Image
from typing import List, Optional, Tuple, Union

from gen_images import parse_range, parse_vec2, make_transform
import legacy
def generate_images(
    network_pkl: str,
    seeds: List[int],
    truncation_psi: float,
    noise_mode: str,
    outdir: str,
    translate: Tuple[float,float],
    rotate: float,
    class_idx: Optional[int]
):
    """Generate images using pretrained network pickle.
    Examples:
    \b
    # Generate an image using pre-trained AFHQv2 model ("Ours" in Figure 1, left).
    python gen_images.py --outdir=out --trunc=1 --seeds=2 \\
        --network=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-afhqv2-512x512.pkl
    \b
    # Generate uncurated images with truncation using the MetFaces-U dataset
    python gen_images.py --outdir=out --trunc=0.7 --seeds=600-605 \\
        --network=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-metfacesu-1024x1024.pkl
    """

    print('Loading networks from "%s"...' % network_pkl)
    device = torch.device('cuda')
    with dnnlib.util.open_url(network_pkl) as f:
        G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore
        #G = legacy.load_network_pkl(f)['G'].to(device) # type: ignore
    os.makedirs(outdir, exist_ok=True)

    # Labels.
    label = torch.zeros([1, G.c_dim], device=device)
    if G.c_dim != 0:
        if class_idx is None:
            raise click.ClickException('Must specify class label with --class when using a conditional network')
        label[:, class_idx] = 1
    else:
        if class_idx is not None:
            print ('warn: --class=lbl ignored when running on an unconditional network')

    # Generate images.
    for seed_idx, seed in enumerate(seeds):
        print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
        z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)

        # Construct an inverse rotation/translation matrix and pass to the generator.  The
        # generator expects this matrix as an inverse to avoid potentially failing numerical
        # operations in the network.
        if hasattr(G.synthesis, 'input'):
            m = make_transform(translate, rotate)
            #print(m)
            m = np.linalg.inv(m)
            #print(m)
            G.synthesis.input.transform.copy_(torch.from_numpy(m))

        img = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
        img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seed:04d}.png')



In [ ]:
!module load cudnn
generate_images("pretrained/stylegan3-t-ffhq-1024x1024.pkl", list(range(19, 19+16)), 0.5, "const", "out2", (0,0), 0, 1)

In [31]:
network_pkl = "pretrained/stylegan3-t-ffhq-1024x1024.pkl"
device = torch.device('cuda')
with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)# ['G_ema'].to(device) # type: ignore
G_b = G['G_ema'].to(device)
seed = 19
z = torch.from_numpy(np.random.RandomState(seed).randn(1, G_b.z_dim)).to(device)
label = torch.zeros([1, G_b.c_dim], device=device)
from torchinfo import summary


In [40]:
inp = {
            "z": z,
            "c": label,
            "truncation_psi": 0.5,
        }
summary(G_b, input_data=inp, depth=5)

Layer (type:depth-idx)                        Output Shape              Param #
Generator                                     [1, 3, 1024, 1024]        --
├─MappingNetwork: 1-1                         [1, 16, 512]              --
│    └─FullyConnectedLayer: 2-1               [1, 512]                  (262,656)
│    └─FullyConnectedLayer: 2-2               [1, 512]                  (262,656)
├─SynthesisNetwork: 1-2                       [1, 3, 1024, 1024]        --
│    └─SynthesisInput: 2-3                    [1, 512, 36, 36]          262,144
│    │    └─FullyConnectedLayer: 3-1          [1, 4]                    (2,052)
│    └─SynthesisLayer: 2-4                    [1, 512, 36, 36]          2,359,808
│    │    └─FullyConnectedLayer: 3-2          [1, 512]                  (262,656)
│    └─SynthesisLayer: 2-5                    [1, 512, 36, 36]          2,359,808
│    │    └─FullyConnectedLayer: 3-3          [1, 512]                  (262,656)
│    └─SynthesisLayer: 2-6                 

In [41]:
G_b

Generator(
  (synthesis): SynthesisNetwork(
    w_dim=512, num_ws=16,
    img_resolution=1024, img_channels=3,
    num_layers=14, num_critical=2,
    margin_size=10, num_fp16_res=4
    (input): SynthesisInput(
      w_dim=512, channels=512, size=[36, 36],
      sampling_rate=16, bandwidth=2
      (affine): FullyConnectedLayer(in_features=512, out_features=4, activation=linear)
    )
    (L0_36_512): SynthesisLayer(
      w_dim=512, is_torgb=False,
      is_critically_sampled=False, use_fp16=False,
      in_sampling_rate=16, out_sampling_rate=16,
      in_cutoff=2, out_cutoff=2,
      in_half_width=6, out_half_width=6,
      in_size=[36, 36], out_size=[36, 36],
      in_channels=512, out_channels=512
      (affine): FullyConnectedLayer(in_features=512, out_features=512, activation=linear)
    )
    (L1_36_512): SynthesisLayer(
      w_dim=512, is_torgb=False,
      is_critically_sampled=False, use_fp16=False,
      in_sampling_rate=16, out_sampling_rate=16,
      in_cutoff=2, out_cutoff

In [8]:
import torch
import torchvision
import PIL
folder = "out2"
def load_and_make_grid(folder):
    images = []
    for filename in sorted(os.listdir(folder), key=lambda x: int(x.split(".")[0].split("seed")[1])):
        img = PIL.Image.open(os.path.join(folder,filename))
        if img is not None:
            images.append(torchvision.transforms.ToTensor()(img))
    return torchvision.utils.make_grid(images, nrow=4)
grid = load_and_make_grid(folder)
torchvision.utils.save_image(grid, "grid_sg3_noema.jpg")